In [150]:
import requests
import pandas as pd

def get_matches_for_league(league_shortcut: str, season: int = 2025):
    """
    Lädt alle Spiele einer Liga (1., 2. oder 3. Bundesliga) von OpenLigaDB.
    league_shortcut: 'bl1', 'bl2', 'bl3'
    season: Jahr der Saison (z. B. 2024 für Saison 2024/25)
    """
    urlOpenLigaDB = f"https://api.openligadb.de/getmatchdata/{league_shortcut}/{season}"
    r = requests.get(urlOpenLigaDB)
    data = r.json()

    matches = []
    for match in data:
        matches.append({
            "league": league_shortcut,
            "match_id": match["matchID"],
            "date": match["matchDateTime"],
            "matchday": match["group"]["groupName"],
            "team_home": match["team1"]["teamName"],
            "team_away": match["team2"]["teamName"],
            #"city": match["location"]["locationCity"],
            #"stadium": match["location"]["locationStadium"]
        })
    return pd.DataFrame(matches)


# Abruf für 1.–3. Bundesliga
df_matches_bl1 = get_matches_for_league("bl1", 2025)
df_matches_bl2 = get_matches_for_league("bl2", 2025)
df_matches_bl3 = get_matches_for_league("bl3", 2025)

# Zusammenführen
df_matches = pd.concat([df_matches_bl1, df_matches_bl2, df_matches_bl3], ignore_index=True)


In [151]:
from bs4 import BeautifulSoup
def scrape_transfermarkt_stadiums(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, "html.parser")

    data = []

    for row in soup.select("table.items > tbody > tr"):
        # optional: nur echte Datenzeilen, klassisch 'odd' oder 'even'
        if 'class' in row.attrs and not any(c in ['odd', 'even'] for c in row['class']):
            continue

        cols = row.find_all("td")
        if len(cols) >= 5:
            # Teamname aus img title
            team_img = cols[1].find("img")
            team = team_img['title'].strip() if team_img else cols[1].get_text(strip=True)

            # Stadion, Ort, Kapazität
            stadion = cols[2].get_text(strip=True)
            ort = cols[3].get_text(strip=True)
            kapaz_raw = cols[4].get_text(strip=True).replace(".", "")
            
            # Nur Zeilen mit gültiger Kapazität übernehmen
            try:
                kapaz = int(kapaz_raw)
                if kapaz < 1000:  # Filter: reale Stadien > 1000 Plätze
                    continue
            except ValueError:
                continue

            data.append([team, stadion, ort, kapaz])

    return pd.DataFrame(data, columns=["team", "stadium", "city", "capacity"])


urls = {
    "1. Bundesliga": "https://www.transfermarkt.de/bundesliga/stadien/wettbewerb/L1",
    "2. Bundesliga": "https://www.transfermarkt.de/2-bundesliga/stadien/wettbewerb/L2",
    "3. Liga": "https://www.transfermarkt.de/3-liga/stadien/wettbewerb/L3"
}

df_stadiums_list = []
for liga, url in urls.items():
    df = scrape_transfermarkt_stadiums(url)
    df_stadiums_list.append(df)

df_stadiums = pd.concat(df_stadiums_list, ignore_index=True)


In [ ]:
# Add location of nearest train station

from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="bundesliga-bahn-mapper")

def find_nearest_station(city):
    try:
        query = f"{city} Hauptbahnhof"
        location = geolocator.geocode(query)
        if location:
            return {
                "station_name": query,
                "lat": location.latitude,
                "lon": location.longitude
            }
        else:
            # fallback: Stadtzentrum
            city_loc = geolocator.geocode(city)
            if city_loc:
                return {
                    "station_name": f"{city} (Zentrum)",
                    "lat": city_loc.latitude,
                    "lon": city_loc.longitude
                }
    except Exception as e:
        print(f"Fehler bei {city}: {e}")
    return None

# Alle einzigartigen Städte extrahieren
cities = pd.Series(df_stadiums["city"].dropna().unique()).tolist()

# Koordinaten sammeln
station_data = {}
for city in cities:
    result = find_nearest_station(city)
    if result:
        station_data[city] = result
    time.sleep(1)  # WICHTIG: Rate-Limit der OSM-API

# Ergebnisse in DataFrame-Form
stations_df = pd.DataFrame.from_dict(station_data, orient="index")
print(stations_df.head())


In [136]:
from fuzzywuzzy import process

# Dictionary: Stadium-Teamname → gewünschtes Match-Name
mapping_teamNames = {}
for m in df_matches["team_home"].unique():
    match_best = process.extractOne(m, df_stadiums["team"].unique())
    mapping_teamNames[match_best[0]] = m

# df_stadiums direkt ersetzen oder neue Spalte erzeugen
df_stadiums["team_clean"] = df_stadiums["team"].replace(mapping_teamNames)

# merge dfs
df_merged1 = df_matches.merge(df_stadiums, how = "left", left_on="team_home", right_on="team_clean", indicator=True)

# add away city
df_merged2 = df_merged1.merge(df_stadiums[["team_clean", "city"]].rename(columns = {"city":"away_city"}), how = "left", left_on="team_away", right_on="team_clean")

# Überprüfen
## df_merged[df_merged["_merge"] == "left_only"]["team_home"].unique()
## print(df_merged["_merge"].value_counts())

# Select Spalten
df_full = df_merged2[['match_id', 'league', 'date', 'matchday', 'team_home', 'team_away', 'stadium', 'capacity', 'city', 'away_city']].rename(columns = {'city':"home_city"})


In [ ]:
# only select 14. Spieltag (for testing)
df_try = df_full[df_full["matchday"] == "14. Spieltag"]

,station_name,lat,lon
Berlin,Berlin Hauptbahnhof,52.525017,13.369448
Frankfurt (Main),Frankfurt (Main) Hauptbahnhof,50.106654,8.662581
Mönchengladbach,Mönchengladbach Hauptbahnhof,51.163220,6.439669
Hamburg,Hamburg Hauptbahnhof,53.553199,10.006436
Sinsheim,Sinsheim Hauptbahnhof,49.250168,8.875500
